In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [2]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (textid);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [5]:
workdir = r'/home/tari/Загрузки/taiga/Magazines'
filename = 'Magazines.db'
file_path = os.path.join(workdir, filename)
metatablepathlist = sorted([os.path.join(workdir,file) for file in os.listdir(workdir) if 'metadata' in file])
mags = [x[0] for x in os.walk(workdir)]
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepathlist[0], sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,segment,textid,textname,textregion,textrubric,textdiff,author,authortexts,authorreaders,magazine,date,time,tags,source
0,Журнальный зал,1709,Последний имажинист,,Маяковский и его время,,,,,Арион,"1995, 1",,,http://magazines.russ.ru/arion/1995/1/8_anna1....
1,Журнальный зал,1711,Ларец и ключ,,Вступительное слово Елены Шварц,,,,,Арион,"1995, 2",,,http://magazines.russ.ru/arion/1995/2/ottisp1....
2,Журнальный зал,1712,Я просто буду рядом,,Вступительное слово Е. Бунимовича,,,,,Арион,"1995, 2",,,http://magazines.russ.ru/arion/1995/2/panteo1....
3,Журнальный зал,1717,Стихи и письма,,Вступительное слово Любови Пустильник,,,,,Арион,"1995, 3",,,http://magazines.russ.ru/arion/1995/3/annals3....
4,Журнальный зал,1720,Хлеб поэзии в век разброда,,Интервью с Иосифом Бродским,,,,,Арион,"1995, 3",,,http://magazines.russ.ru/arion/1995/3/dialog.html


In [6]:
meta.columns

Index(['segment', 'textid', 'textname', 'textregion', 'textrubric', 'textdiff',
       'author', 'authortexts', 'authorreaders', 'magazine', 'date', 'time',
       'tags', 'source'],
      dtype='object')

In [7]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [8]:
meta.iloc[7].to_dict()

{'author': '',
 'authorreaders': '',
 'authortexts': '',
 'date': '1996, 2',
 'magazine': 'Арион',
 'segment': 'Журнальный зал',
 'source': 'http://magazines.russ.ru/arion/1996/2/arion3.html',
 'tags': '',
 'textdiff': '',
 'textid': 1814,
 'textname': 'Неизвестные стихи',
 'textregion': '',
 'textrubric': 'Вступительное слово Лазаря Шерешевского. Публикация Н. Н. Глазкова',
 'time': ''}

In [9]:
#meta = meta.drop_duplicates(subset=None, keep='first', inplace=False)
meta.textid.value_counts()[:10]

2045      1
81033     1
102575    1
1709      1
46251     1
46247     1
2211      1
2207      1
106648    1
106645    1
Name: textid, dtype: int64

In [ ]:
for ind in range(len(metatablepathlist)):
    print(ind, len(metatablepathlist))
    meta = pd.read_csv(metatablepathlist[ind], sep='\t', encoding='utf8')
    meta = meta.fillna('')
    meta = meta.drop_duplicates(subset=None, keep='first', inplace=False)
    for i in tqdm(range(len(meta))):
        values = meta.iloc[i].to_dict()
        values['textid'] = str(values['textid'])

        columns = ', '.join(values.keys())

        #print(list(values.values()))
        placeholders = ', '.join('?' * len(values))
        sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
        #print(sql)
        try:
            cur.execute(sql, list(values.values()))

            valuest = {'id': values['textid'], 'text': unify.open_text(os.path.join(os.path.join(plain,years[ind]), str(values['textid'])+".txt"))}

            columns = ', '.join(valuest.keys())
            placeholders = ', '.join('?' * len(valuest))

            sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
            cur.execute(sql2, list(valuest.values()))
            try:
                valuest2 = {'id': values['textid'], 'text': unify.open_text(open(os.path.join(os.path.join(tagged,years[ind]), str(values['textid'])+".txt"),'r', encoding='utf8').read())}
                columns = ', '.join(valuest2.keys())
                placeholders = ', '.join('?' * len(valuest2))
                sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
                cur.execute(sql3, list(valuest2.values()))
            except:
                valuest2 = {'id': values['textid'], 'text': ""}
                columns = ', '.join(valuest2.keys())
                placeholders = ', '.join('?' * len(valuest2))
                sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
                cur.execute(sql3, list(valuest2.values()))
            con.commit()
        except:
            continue